<h1>human organ classification </h1>
<h2> C-NN model<h3> [end to end model]</h3></h2>

call to packages & libraries

In [1]:
import sys
import os
from tensorflow import keras
from datetime import datetime

from tensorflow.keras.optimizers import RMSprop,SGD,Adam
from tensorflow.keras.layers import Dense,BatchNormalization,Dropout, GlobalMaxPooling2D
from tensorflow.keras.layers import Conv2D,MaxPool2D,Flatten
from tensorflow.keras.applications import VGG19
from tensorflow.keras import Model

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"\\sup")

from evaluation import *
from support import *
from test_set_eval import *

In [2]:
model_name = "VGG19_tl"

# Load the TensorBoard notebook extension.
%load_ext tensorboard

# call inline plt.

In [3]:
# Clear any logs from previous runs
!rm -rf ./logs/

'rm' is not recognized as an internal or external command,
operable program or batch file.


callout dataset

In [3]:
classes = ['dyed-lifted-polyps', 'dyed-resection-margins', 'esophagitis', 'normal-cecum', 'normal-pylorus',
           'normal-z-line', 'polyps', 'ulcerative-colitis']
root_dir = '../../data/kvasir-dataset-v2'

take a glance at training dataset

In [ ]:
plot_sample_of_img(4,4,os.listdir(tr_heart_dir)+os.listdir(tr_eye_dir))

ImageGenator - autolabelling, and categorizing.

In [4]:
train_gen_tmp = ImageDataGenerator(rescale=1. / 255,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest',
                                   rotation_range=60,
                                   validation_split=0.2)

train_gen = train_gen_tmp.flow_from_directory(root_dir,
                                              target_size=(224, 224),
                                              color_mode='rgb',
                                              class_mode='categorical',
                                              batch_size=40,
                                              shuffle=True,
                                              seed=42,
                                              subset='training')

validation_gen = train_gen_tmp.flow_from_directory(root_dir,
                                                   target_size=(224, 224),
                                                   color_mode='rgb',
                                                   class_mode='categorical',
                                                   batch_size=40,
                                                   shuffle=False,
                                                   seed=42,
                                                   subset='validation')

STEP_SIZE_TRAIN = train_gen.n // train_gen.batch_size
STEP_SIZE_VALID = validation_gen.n // validation_gen.batch_size

clToInt_dict = train_gen.class_indices
clToInt_dict = dict((k, v) for v, k in clToInt_dict.items())

Found 6400 images belonging to 8 classes.
Found 1600 images belonging to 8 classes.


define the model

In [6]:
pre_model = VGG19(input_shape=(224,224,3),
                    weights = 'imagenet',
                    include_top = True)
pre_model.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [5]:
pre_model = VGG19(input_shape=(224,224,3),
                    weights = 'imagenet',
                    include_top = True)

#model.summary()

for layer in pre_model.layers:
    layer.trainable = False

conn_layer = pre_model.get_layer('block5_pool')
conn_output = conn_layer.output


x = GlobalMaxPooling2D()(conn_output)
x = BatchNormalization()(x)
x = Dense(128,activation='relu')(x)
#x = Dropout(0.2)(x)
#
x = Dense(8,activation='softmax')(x)

model = Model(pre_model.input,x)

model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

compile the model

In [6]:
model.compile(optimizer=Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

save the log

In [7]:

# Define the Keras TensorBoard callback.
logdir="../../logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")+'/'

if not os.path.exists(logdir):
    os.mkdir(logdir)

#print(datetime.now().strftime("%Y%m%d%H%M%S"))
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir, histogram_freq=1, profile_batch = 100000000)

fit & train the model.

In [8]:
history = model.fit(train_gen,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=validation_gen,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=15,
                    verbose=1,
                    callbacks= [tensorboard_callback])

Epoch 1/15
  7/160 [>.............................] - ETA: 16:07 - loss: 2.3791 - accuracy: 0.1714

KeyboardInterrupt: 

visualize layer process in cnn

In [ ]:
#visualize_model(model,img_path)

instant evaluation

In [ ]:
#call to the tensorboard
%tensorboard --logdir logs
#look at training model performance
acc_n_loss(history)

model.evaluate_generator(validation_gen,
                         steps=STEP_SIZE_VALID)

evaluate the model on test set.

In [ ]:
y_pred,y_test = test_eval(model,classes)
plot_confusion_metrix(y_test,y_pred,classes)
ROC_classes(6,y_test,y_pred,classes)



save the model in .h5 file

In [ ]:
model_path,model_weight_path = save(model,datetime.now()+model_name)

make prediction on random images

In [ ]:
#rnd_predict(model_path,model_weight_path,img_path,clToInt_dict)
